# Proyecto de Python learning

## Librerias

In [83]:
import kagglehub
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings("ignore")



## Importar data

In [3]:
path = kagglehub.dataset_download("emonsharkar/python-learning-and-exam-performance-dataset")
df = pd.read_csv(os.path.join(path, os.listdir(path)[0]))
df.head()


,student_id,age,country,prior_programming_experience,weeks_in_course,hours_spent_learning_per_week,practice_problems_solved,projects_completed,tutorial_videos_watched,uses_kaggle,participates_in_discussion_forums,debugging_sessions_per_week,self_reported_confidence_python,final_exam_score,passed_exam
0,1,54,Pakistan,Beginner,12,12.2,60,1,24,0,1,4,10,53.8,0
1,2,44,Nigeria,NaN,15,14.2,52,1,32,0,0,5,7,31.9,0
2,3,30,India,Intermediate,5,7.7,71,1,43,0,0,6,3,59.4,0
3,4,23,Nigeria,Beginner,4,5.6,64,2,40,1,0,3,10,58.8,0
4,5,36,UK,NaN,3,3.5,62,1,47,1,0,3,7,24.8,0


In [4]:
df.shape

(3000, 15)

In [5]:
df.isnull().sum()

student_id                              0
age                                     0
country                                 0
prior_programming_experience         1062
weeks_in_course                         0
hours_spent_learning_per_week           0
practice_problems_solved                0
projects_completed                      0
tutorial_videos_watched                 0
uses_kaggle                             0
participates_in_discussion_forums       0
debugging_sessions_per_week             0
self_reported_confidence_python         0
final_exam_score                        0
passed_exam                             0
dtype: int64

In [6]:
df.describe(include = "all").round(0).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
student_id,3000.0,NaN,NaN,NaN,1500.0,866.0,1.0,751.0,1500.0,2250.0,3000.0
age,3000.0,NaN,NaN,NaN,35.0,11.0,16.0,26.0,36.0,45.0,54.0
country,3000,10,Brazil,339,NaN,NaN,NaN,NaN,NaN,NaN,NaN
prior_programming_experience,1938,3,Beginner,1034,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weeks_in_course,3000.0,NaN,NaN,NaN,8.0,4.0,1.0,5.0,8.0,12.0,15.0
hours_spent_learning_per_week,3000.0,NaN,NaN,NaN,7.0,3.0,0.0,5.0,7.0,9.0,17.0
practice_problems_solved,3000.0,NaN,NaN,NaN,60.0,8.0,35.0,55.0,60.0,65.0,87.0
projects_completed,3000.0,NaN,NaN,NaN,2.0,1.0,0.0,1.0,2.0,3.0,9.0
tutorial_videos_watched,3000.0,NaN,NaN,NaN,40.0,6.0,19.0,35.0,40.0,44.0,63.0
uses_kaggle,3000.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   student_id                         3000 non-null   int64  
 1   age                                3000 non-null   int64  
 2   country                            3000 non-null   object 
 3   prior_programming_experience       1938 non-null   object 
 4   weeks_in_course                    3000 non-null   int64  
 5   hours_spent_learning_per_week      3000 non-null   float64
 6   practice_problems_solved           3000 non-null   int64  
 7   projects_completed                 3000 non-null   int64  
 8   tutorial_videos_watched            3000 non-null   int64  
 9   uses_kaggle                        3000 non-null   int64  
 10  participates_in_discussion_forums  3000 non-null   int64  
 11  debugging_sessions_per_week        3000 non-null   int64

## Pipeline

### Creacion de Pipes

#### Clasico

In [81]:
drop_cols = ["prior_programming_experience", "student_id", "final_exam_score"]


num_var = make_column_selector(dtype_include="number")
cat_var = make_column_selector(dtype_include="object")

pipe_drop = FunctionTransformer(
                lambda x: x.drop(drop_cols, axis = 1, errors = "ignore"),
                #feature_names_out="one-to-one"
                                )

pipe_cat = Pipeline(steps = [
                    ("cat_imputer", SimpleImputer(strategy= "most_frequent")),
                    ("ohe", OneHotEncoder()),
                    ]
                    )

pip_num = Pipeline(steps = [("imputer", SimpleImputer(strategy = "median"))])

pp = ColumnTransformer( transformers= [
    #("drop", "drop", drop_cols), Si lo hubiera considerado aquí hubiera hecho en paralelo la eliminacion y luego la categorizacion, por lo que habria considerado igual las clumnas eliminadas
    ("categ", pipe_cat, cat_var),
    ("num", pip_num, num_var)],
    remainder= "passthrough"
)

pipe_final = Pipeline(steps = [

                ("drop", pipe_drop),
                ("preprocessor", pp),
                ("model", LogisticRegression(max_iter= 10000))

])

target = "passed_exam"

df_x = df.drop(target, axis = 1)
df_y = df[target]

x_train, x_test , y_train, y_test = train_test_split(df_x, df_y, test_size= 0.5, random_state= 7)

pipe_final.fit(x_train, y_train)
y_predict = pipe_final.predict(x_test )

acc = accuracy_score(y_test, y_predict)
f1 = f1_score(y_test, y_predict)
recall = recall_score(y_test, y_predict)
precision = precision_score(y_test, y_predict)


print("Métricas de desempeño - Regresión Logística")
print("-"*50)
print(f"Accuracy  : {acc:.3f}")
print(f"F1 Score  : {f1:.3f}")
print(f"Recall    : {recall:.3f}")
print(f"Precision : {precision:.3f}")
print("-"*50)





Métricas de desempeño - Regresión Logística
--------------------------------------------------
Accuracy  : 0.837
F1 Score  : 0.350
Recall    : 0.253
Precision : 0.569
--------------------------------------------------


In [53]:
x_train_pp = pipe_final[:-1].transform(x_train) # Arreglo de numpy sin nombre_columnas
nombre_cols = pipe_final.named_steps["preprocessor"].get_feature_names_out()
nombre_cols
df_train_pipe = pd.DataFrame(x_train_pp, columns = nombre_cols )
df_train_pipe

,categ__country_Bangladesh,categ__country_Brazil,categ__country_Germany,categ__country_India,categ__country_Indonesia,categ__country_Nigeria,categ__country_Other,categ__country_Pakistan,categ__country_UK,categ__country_USA,num__age,num__weeks_in_course,num__hours_spent_learning_per_week,num__practice_problems_solved,num__projects_completed,num__tutorial_videos_watched,num__uses_kaggle,num__participates_in_discussion_forums,num__debugging_sessions_per_week,num__self_reported_confidence_python
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,23.0,7.0,2.0,61.0,4.0,37.0,0.0,1.0,6.0,3.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,30.0,3.0,4.4,59.0,2.0,40.0,0.0,1.0,4.0,2.0
2,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,35.0,6.0,7.1,76.0,3.0,26.0,1.0,0.0,5.0,2.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,49.0,5.0,10.5,72.0,3.0,43.0,0.0,1.0,3.0,5.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,49.0,10.0,13.5,71.0,3.0,34.0,1.0,0.0,2.0,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,38.0,6.0,6.0,60.0,4.0,51.0,1.0,0.0,5.0,10.0
1496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,21.0,11.0,7.4,55.0,3.0,28.0,0.0,0.0,4.0,7.0
1497,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,25.0,3.0,4.9,60.0,3.0,35.0,1.0,1.0,5.0,5.0
1498,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.0,15.0,5.6,83.0,3.0,46.0,0.0,0.0,4.0,8.0


In [ ]:
pipe_final.named_steps["preprocessor"].get_feature_names_out()

In [41]:
# 1. Obtener nombres de features después del preprocesamiento
feature_names = pipe_final.named_steps["preprocessor"].get_feature_names_out()

# 2. Obtener coeficientes del modelo entrenado
coefs = pipe_final.named_steps["model"].coef_[0]

# 3. Imprimir
print("\n📌 Coeficientes de la Regresión Logística")
for name, coef in zip(feature_names, coefs):
    print(f"{name:30s} {coef:.5f}")

# Intercepto
intercept = pipe_final.named_steps["model"].intercept_[0]
print("\nIntercepto:", round(intercept, 5))


📌 Coeficientes de la Regresión Logística
categ__country_Bangladesh      0.00644
categ__country_Brazil          0.31778
categ__country_Germany         0.45085
categ__country_India           -0.04325
categ__country_Indonesia       -0.08368
categ__country_Nigeria         -0.27729
categ__country_Other           -0.25073
categ__country_Pakistan        -0.03669
categ__country_UK              -0.15887
categ__country_USA             0.15332
num__age                       0.00748
num__weeks_in_course           -0.00610
num__hours_spent_learning_per_week 0.28417
num__practice_problems_solved  0.02319
num__projects_completed        0.53355
num__tutorial_videos_watched   -0.00388
num__uses_kaggle               0.52897
num__participates_in_discussion_forums 0.14076
num__debugging_sessions_per_week 0.14343
num__self_reported_confidence_python 0.23945

Intercepto: -8.89247


#### Stepwise

In [93]:
drop_cols = ["prior_programming_experience", "student_id", "final_exam_score"]


num_var = make_column_selector(dtype_include="number")
cat_var = make_column_selector(dtype_include="object")

pipe_drop = FunctionTransformer(
                lambda x: x.drop(drop_cols, axis = 1, errors = "ignore")
                                )

pipe_cat = Pipeline(steps = [
                    ("cat_imputer", SimpleImputer(strategy= "most_frequent")),
                    ("ohe", OneHotEncoder()),
                    ]
                    )

pip_num = Pipeline(steps = [("imputer", SimpleImputer(strategy = "median"))])

pp = ColumnTransformer( transformers= [
    #("drop", "drop", drop_cols),
    ("categ", pipe_cat, cat_var),
    ("num", pip_num, num_var)],
    remainder= "passthrough"
)
#######################################################################################
sfs = SequentialFeatureSelector(
    estimator=LogisticRegression(),
    n_features_to_select=8,          # <-- define cuántas features finales quieres
    direction="backward",
    scoring="accuracy",                # o "f1", "accuracy", etc.
    #cv=StratifiedKFold(5, shuffle=True, random_state=42),
    n_jobs=-1
)


#######################################################################################

pipe_final = Pipeline(steps = [

                ("drop", pipe_drop),
                ("preprocessor", pp),
                ("backward", sfs),
                ("model", LogisticRegression(max_iter= 10000))

])

target = "passed_exam"

df_x = df.drop(target, axis = 1)
df_y = df[target]

x_train, x_test , y_train, y_test = train_test_split(df_x, df_y, test_size= 0.5, random_state= 7)

pipe_final.fit(x_train, y_train)
y_predict = pipe_final.predict(x_test )

acc = accuracy_score(y_test, y_predict)
f1 = f1_score(y_test, y_predict)
recall = recall_score(y_test, y_predict)
precision = precision_score(y_test, y_predict)


print("Métricas de desempeño - Regresión Logística")
print("-"*50)
print(f"Accuracy  : {acc:.3f}")
print(f"F1 Score  : {f1:.3f}")
print(f"Recall    : {recall:.3f}")
print(f"Precision : {precision:.3f}")
print("-"*50)


Métricas de desempeño - Regresión Logística
--------------------------------------------------
Accuracy  : 0.838
F1 Score  : 0.315
Recall    : 0.215
Precision : 0.596
--------------------------------------------------


In [66]:
pipe_final.named_steps["preprocessor"].get_feature_names_out()[pipe_final.named_steps["backward"].get_support()]

array(['categ__country_Nigeria', 'categ__country_Other',
       'categ__country_UK', 'num__age',
       'num__hours_spent_learning_per_week',
       'num__practice_problems_solved', 'num__projects_completed',
       'num__self_reported_confidence_python'], dtype=object)

In [13]:
pipe_final.named_steps["preprocessor"].get_feature_names_out()[pipe_final.named_steps["backward"].get_support()]

array(['categ__country_Nigeria', 'categ__country_Other',
       'categ__country_UK', 'num__age',
       'num__hours_spent_learning_per_week',
       'num__practice_problems_solved', 'num__projects_completed',
       'num__self_reported_confidence_python'], dtype=object)

In [14]:
# 1) Obtener el modelo ya entrenado desde el pipeline
model = pipe_final.named_steps["model"]

# 2) Obtener los nombres de las features tras el preprocesamiento
feature_names = pipe_final.named_steps["backward"].get_feature_names_out()

# 3) Obtener los coeficientes (vector 1D)
coefs = model.coef_[0]
intercept = model.intercept_[0]

# 4) Imprimir coeficientes + intercepto
print("\n📌 Coeficientes del modelo logístico:")
for name, coef in zip(feature_names, coefs):
    print(f"{name:30s} {coef:.5f}")

print(f"\nIntercepto: {intercept:.5f}")



📌 Coeficientes del modelo logístico:
x5                             -0.41477
x6                             -0.38912
x8                             -0.32141
x10                            0.00632
x12                            0.27398
x13                            0.02233
x14                            0.49614
x19                            0.22407

Intercepto: -7.58760


In [15]:
import numpy as np
import pandas as pd

pre = pipe_final.named_steps["preprocessor"]
sfs = pipe_final.named_steps["backward"]
lr  = pipe_final.named_steps["model"]

# 1) Todos los nombres después del preprocesamiento (M)
all_names = pre.get_feature_names_out()


# 2) Máscara de selección del SFS (M -> k)
mask = sfs.get_support()



# 3) Nombres seleccionados (k)
sel_names = all_names[mask]

print(sel_names)

# 4) Coefs del modelo (k)
coefs = lr.coef_.ravel()
intercept = lr.intercept_[0]

# 5) Tabla ordenada
df_coef = pd.DataFrame({"feature": sel_names, "coef": coefs})
df_coef["abs_coef"] = df_coef["coef"].abs()
df_coef = df_coef.sort_values("abs_coef", ascending=False)

print("\n📌 Coeficientes (post-backward SFS):")
print(df_coef[["feature","coef"]].to_string(index=False))
print("\nIntercepto:", intercept)


['categ__country_Nigeria' 'categ__country_Other' 'categ__country_UK'
 'num__age' 'num__hours_spent_learning_per_week'
 'num__practice_problems_solved' 'num__projects_completed'
 'num__self_reported_confidence_python']

📌 Coeficientes (post-backward SFS):
                             feature      coef
             num__projects_completed  0.496137
              categ__country_Nigeria -0.414773
                categ__country_Other -0.389122
                   categ__country_UK -0.321414
  num__hours_spent_learning_per_week  0.273980
num__self_reported_confidence_python  0.224074
       num__practice_problems_solved  0.022327
                            num__age  0.006315

Intercepto: -7.587600628009373


In [139]:
feature_names = sel_names
coefs = pipe_final.named_steps["model"].coef_[0]
intercept = pipe_final.named_steps["model"].intercept_[0]

# Construir fórmula
terms = " + ".join([f"({coef:.5f} * {name})" for name, coef in zip(feature_names, coefs)])
formula = f"logit(p) = {intercept:.5f} + {terms}"

print("\n🧮 Fórmula final del modelo:")
print(formula)

print("\n🔹 Probabilidad:")
print("p = 1 / (1 + exp(-logit(p)))")



🧮 Fórmula final del modelo:
logit(p) = -200.02859 + (0.60801 * categ__country_Brazil) + (0.64138 * categ__country_Nigeria) + (-0.14407 * categ__country_Other) + (-0.22768 * categ__country_UK) + (-0.39909 * categ__country_USA) + (-0.03134 * num__weeks_in_course) + (0.38554 * num__uses_kaggle) + (3.33309 * num__final_exam_score)

🔹 Probabilidad:
p = 1 / (1 + exp(-logit(p)))


# Gridsearch

In [ ]:
drop_cols = ["prior_programming_experience", "student_id", "final_exam_score"]

# num_var = df.select_dtypes("number").columns
# cat_var = df.select_dtypes("object").columns

num_var = make_column_selector(dtype_include="number")
cat_var = make_column_selector(dtype_include="object")

pipe_drop = FunctionTransformer(
        lambda x : x.drop(drop_cols, axis =1, errors = "ignore")
                                )

pipe_num = Pipeline(steps = [("imp_num", SimpleImputer(strategy = "median"))])

pipe_cat = Pipeline(steps = [("imp_cat", SimpleImputer(strategy="most_frequent")),
                             ("ohe", OneHotEncoder())])

pp = ColumnTransformer( transformers= [("num", pipe_num, num_var),
                                            ("cat", pipe_cat, cat_var)],
                                         remainder = "passthrough"
                                )

pipe_classifier = {
    
    "LogisticRegression" : {

        "model": LogisticRegression(),
        "params": {"classifier__C": [0.1,1,10],
                   "classifier__random_state" : [42]
                   }
    },

    "DecisionTree" : {

        "model" : DecisionTreeClassifier(),
        "params": {"classifier__max_depth": [1,2,3,4],
                   "classifier__criterion": ["gini"],
                   "classifier__random_state" : [42]
                   }

    },

    "RandomForest" : {

        "model": RandomForestClassifier(),
        "params": {"classifier__n_estimators": [100, 200,300, 400, 500],
                   "classifier__random_state" : [42]
                   #"classifier__max_depth" : []
                   }

    },

    "XGBoost" : {

        "model": XGBClassifier(),
        "params": {"classifier__n_estimators": [100, 200,300, 400, 500],
                   "classifier__learning_rate" : [0.01, 0.02, 0.03],
                   "classifier__random_state" : [42]
                   }
    },

}

target = "passed_exam"

df_x = df.drop(target, axis = 1)
df_y = df[target]

x_train, x_test , y_train, y_test = train_test_split(df_x, df_y, test_size= 0.5, random_state= 7)

mejor_modelo = {}

for name, modelo_data in pipe_classifier.items():
    pipe_final = Pipeline( steps = [
        ("drop", pipe_drop),
        ("preprocessor",  pp),
        ("classifier", modelo_data["model"])
    ])

    grid = GridSearchCV(pipe_final, modelo_data["params"], scoring = "accuracy", cv = 5)
    grid.fit(x_train, y_train)

    mejor_modelo[name] = {"mejor_accuracy" : grid.best_score_,
                          "mejores_parametros": grid.best_params_,
                          "mejor_estimator": grid.best_estimator_}
    
    print(f"{name} optimizado con Accuracy: {grid.best_score_:.4f}")

ganador = max(mejor_modelo, key=lambda x: mejor_modelo[x]['mejor_accuracy'])
print(f"El mejor modelo es {ganador}")



LogisticRegression optimizado con Accuracy: 0.8253
DecisionTree optimizado con Accuracy: 0.8193
RandomForest optimizado con Accuracy: 0.8333
XGBoost optimizado con Accuracy: 0.8247
LogReg_SFS_Backward optimizado con Accuracy: nan
El mejor modelo es RandomForest
